## 날씨 정보 크롤링 

In [1]:
#requests 크롤링에서 페이지정보 추출하는 라이브러리 
import requests as req 
#1step) 날 것의 데이터 추출하기
url =(' https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=%EB%82%A0%EC%94%A8')
res=req.get(url)
#2step) 자연어를 컴퓨터(html)언어로 바꾸기 위해 BeautifulSoup라이브러리로 데이터를 객체화(html형태데이터화)하기
from bs4 import BeautifulSoup as bs 
#bs(페이지정보, 파싱방법) 이렇게 하면 html언어화 
soup = bs(res.text, 'lxml')
#3step) 태그명과 선택자를 사용하여 가져오기 
weather=soup.select('a.news_tit')
# 주의: 가져온 결과가 리스트 형태로 나와서 인덱싱 후에 .text로 뽑아내기
soup.select('a.news_tit')[0].text 
#위처럼 하면 결과 하나씩 나오게 되며, 여러 개를 한 번에 출력하려면 
#4step) for문 사용하기
for i in range(len(weather)):
    print(weather[i].text)

"나들이 못 가겠네"…전국에 천둥·번개 동반한 비[오늘날씨]
[내일날씨]전국에 천둥·번개 동반한 비…일부 지역 우박
[내일날씨] 천둥·번개 동반 요란한 가을비…일부선 우박
[생활날씨] 주말 국지성 호우…벼락·돌풍·우박 주의


## 멜론차트 크롤링 후 데이터프레임화하기

In [10]:
#step0) 라이브러리 불러오기 
import requests as req 
from bs4 import BeautifulSoup as bs
import pandas as pd 
#step1) 날 것의 데이터 추출하기 
#주의) Response [400]: 브라우저에서 사용자가 아닌 크롤링 코드로 하고 있음을 감지하여 접근불가됨
#Response [200] : 정보를 성공적으로 응답받았다!
#Response [400] : 클라이언트의 요청에 문제가 있을 때 발생하는 오류코드
#Response [500] : 서버의 응답에 문제가 있을 때 발생하는 오류코드
#해결) 객체 형태? 딕셔너리 형태로 
#'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
#이걸 headers에 넣어주기
url ='https://www.melon.com/chart/'
h={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
res=req.get(url, headers = h)
#step3) BeautifulSoup 라이브러리로 자연어를 컴퓨터가 인식하도록 객체화(html형태데이터화)
soup = bs(res.text , 'lxml')
#step4) 태그와 선택자명으로 노래제목 및 가수 파싱하기 
song=soup.select('div.ellipsis.rank01>span>a')
#주의)가수명이 두 개 이상인 경우, 각 가수가 a태그로 따로 되어있어서, a태그로 파싱하면
#두개로 추출됨 그러면 노래제목과 가수명의 개수가 일치하지 않게된다. 
singer=soup.select('div.ellipsis.rank02>span>a')
#해결)두 가수를 한꺼번에 묶고 있는 태그 span으로 추출하기 
singer=soup.select('div.ellipsis.rank02>span')
#step5)결과가 리스트 형태로 나오므로, for문으로 인덱싱하여, 다 추출하기
song_list = []
singer_list=[]
rank_list=[]
for i in range(len(song)):
    song_list.append(song[i].text)
    singer_list.append(singer[i].text)
    rank_list.append(i+1)

In [11]:
#step6)딕셔너리를 활용해서 DataFrame만들기 
dic = {'노래제목': song_list, "가수": singer_list, "순위": rank_list}
melon=pd.DataFrame(dic)
melon.set_index('순위',inplace=True)
melon

,노래제목,가수
순위,,
1,Love Lee,AKMU (악뮤)
2,후라이의 꿈,AKMU (악뮤)
3,Seven (feat. Latto) - Clean Ver.,정국
4,You & Me,제니 (JENNIE)
5,Either Way,IVE (아이브)
...,...,...
96,무중력 (Space),NCT 127
97,사실말야내가말야그게그러니까말이야,케이시 (Kassy)
98,미친 사랑의 노래,"김연지, DK(디셈버)"


In [12]:
#step7)csv  파일로 내보내기
melon.to_csv('data//멜론차트복습.csv')

### iframe 조심) iframe으로 감싸진 것을 반환할 때는 응답이 정상이 뜨더라도 데이터 출력이 안된다. 
- 원본데이터가 있는 주소로 이동하여 그 주소를 url에 넣기 